In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import LabelEncoder
import numpy as np
from data_work import data_split
from data_work import processed
from data_work import analyse_dataset
from data_work import set_data
from sklearn.model_selection import train_test_split
import pandas as pd
data=pd.read_csv("/Users/mac/Dev/2022_SE03/data/lasted_data.csv",index_col=None)
data=data.drop("Unnamed: 0",axis=1)

In [2]:

traindata,test_data=train_test_split(data,test_size=0.4,random_state=42,stratify=data['subcategory '])
data = traindata.reset_index(drop=True)
test_data = test_data.reset_index(drop=True)
client_data,server_data=train_test_split(traindata,test_size=0.001,random_state=42)
client_data = client_data.reset_index(drop=True)
server_data = server_data.reset_index(drop=True)

In [3]:
trainloaders,testloaders=data_split(client_data,"subcategory ",8,0.1,IID=0)

prepairing non IID dataset
len of client data :4208, len of each subdata : 526
client number 1 : train(474), val(52)
client number 2 : train(474), val(52)
client number 3 : train(474), val(52)
client number 4 : train(474), val(52)
client number 5 : train(474), val(52)
client number 6 : train(474), val(52)
client number 7 : train(474), val(52)
client number 8 : train(474), val(52)


In [4]:
import torch
for i in range(len(trainloaders)):
    trainloaders[i]=torch.tensor(trainloaders[i].values)

In [9]:
from model import Net
from CGan import CGAN
class Argument():
    def __init__(self,data):
        self.noise_size=5
        self.n_features=28
        self.n_classes=8
        self.epoch=10
        self.batch_size=32
        self.dataset=data #tensor
        self.lrG=0.003
        self.lrD=0.003 

In [10]:
class client():
    def __init__(self,cid):
       self.cid=cid
       self.trainset=trainloaders[self.cid] #tensor
       self.testset=testloaders[self.cid] #tensor
        
       args=Argument(self.trainset) #customs

       self.Gan=CGAN(args)
       self.model=Net(0.00)
       
    def update_model(self,params):
        self.model.load_parameters(params)
    def update_Gan(self,params):
        self.Gan.load_parameters(params)
    def Gan_fit(self):
        self.Gan.train()
        #code Generator fit data
    def model_fit(self):
        X=self.trainset[:,1:]
        y=self.trainset[:,:1]
        y=y.squeeze().tolist()
        y=torch.tensor(y)
        self.model.fit(X,y,0.003,0.2,10)
        #model classify fit
    def get_parameters(self):
        D_parameters,G_parameters=self.Gan.get_parameters()
        M_parameters=self.model.get_parameters()
        return M_parameters,D_parameters, G_parameters
    def evaluate(self):
        X=self.testset[:,1:]
        y=self.testset[:,:1]
        y=y.squeeze().tolist()
        y=torch.tensor(y)
        accuracy=self.model.evaluate(X,y)
        return accuracy

In [11]:
def fn_client(cid)->client:
    return client(cid)



In [12]:
client_1=fn_client(1)


---------- Networks architecture -------------
Gen(
  (label_emb): Embedding(8, 8)
  (model): Sequential(
    (0): Linear(in_features=13, out_features=128, bias=True)
    (1): LeakyReLU(negative_slope=0.2)
    (2): Linear(in_features=128, out_features=256, bias=True)
    (3): BatchNorm1d(256, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
    (4): LeakyReLU(negative_slope=0.2)
    (5): Linear(in_features=256, out_features=512, bias=True)
    (6): BatchNorm1d(512, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
    (7): LeakyReLU(negative_slope=0.2)
    (8): Linear(in_features=512, out_features=1028, bias=True)
    (9): BatchNorm1d(1028, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
    (10): LeakyReLU(negative_slope=0.2)
    (11): Linear(in_features=1028, out_features=28, bias=True)
    (12): BatchNorm1d(28, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
    (13): LeakyReLU(negative_slope=0.2)
  )
)
Total number of parameters:

TypeError: Net.__init__() missing 1 required positional argument: 'dropout_rate'

In [ ]:
class server():
    def __init__(self,args):
        self.model=Net()
        self.Gan=CGAN()
        self.clients=[]
        for i in range(number_clients):
            cl=fn_client(i)
            self.clients.append(cl)
        self.trainset=server_data #tensor
        self.testset=test_data #tensor
        
        


SyntaxError: incomplete input (2978428367.py, line 1)